In [1]:
# this script is used to read from the cases metadata and enrich them with the judges metadata
# the cases metadata start out wth caseid, case_reversed, judge_id, year and log_cites
# the judges metadata list LastName, FirstName, Gender, CourtType, nominating Presidents and their party affiliation

import pandas as pd
import numpy as np


In [2]:
CASES_MD = "/home/xhta/Robot/case_metadata.csv"
JUDGES_MD = "/home/xhta/Robot/judges/judges-7.csv"


In [3]:
md = pd.DataFrame(index = range(0,0), columns=['caseid','case_reversed','judge_id','year','log_cites'])

md = pd.read_csv (CASES_MD)
md["case_reversed"] = md["case_reversed"].fillna(0).astype(int)
md["judge_id"] = md["judge_id"].fillna(0).astype(int)
md["year"] = md["year"].fillna(0).astype(int)
md["log_cites"] = md["log_cites"].fillna(0).astype(float)


In [4]:
empty_str = '' * len(md)
md["LastName"] = pd.Series(empty_str, index = md.index)
md["FirstName"] = pd.Series(empty_str, index = md.index)
md["Gender"] = pd.Series([-1]*len(md), index = md.index)
md["Pres"] = pd.Series(empty_str, index = md.index)
md["Party"] = pd.Series(empty_str, index = md.index)

md["log_cites"] = md["log_cites"].fillna(0)

for col in ['LastName', 'FirstName', 'Pres', 'Party']:
    md[col] = md[col].astype(object)

md["Gender"] = md["Gender"].astype(int)


In [5]:
md.head()

,caseid,case_reversed,judge_id,year,log_cites,LastName,FirstName,Gender,Pres,Party
0,X3JGGO,0,1653,1925,1.098612,,,-1,,
1,X3OH3J,0,1034,1924,1.609438,,,-1,,
2,X3U0KO,0,2303,1925,1.791759,,,-1,,
3,X53HAD,0,1764,1924,0.000000,,,-1,,
4,X9VC5V,0,493,1925,0.000000,,,-1,,


In [6]:
ju = pd.read_csv(JUDGES_MD, sep=';', encoding='cp1252')
ju.columns = ["judge_id", "LastName", "FirstName", "Gender", "CourtType", "Pres", "Party"]
#
ju["judge_id"] = ju["judge_id"].fillna(0).astype(int)
#
ju.loc[ ju.Gender  == 'Female', 'Gender' ] = 0
ju.loc[ ju.Gender  == 'Male', 'Gender' ] = 1


In [7]:
md = md.set_index("caseid")
ju = ju.set_index("judge_id")


In [8]:
for inde in md.index:
    jid = md.loc[inde, "judge_id"]
    if jid not in ju.index: continue
    md.loc[inde, "LastName"] = ju.loc[jid, "LastName"]
    md.loc[inde, "FirstName"] = ju.loc[jid, "FirstName"]
    md.loc[inde, "Gender"] = ju.loc[jid, "Gender"]
    md.loc[inde, "Pres"] = ju.loc[jid, "Pres"]
    md.loc[inde, "Party"] = ju.loc[jid, "Party"]


In [9]:
md.head()

,case_reversed,judge_id,year,log_cites,LastName,FirstName,Gender,Pres,Party
caseid,,,,,,,,,
X3JGGO,0,1653,1925,1.098612,Miller,Wilbur,1,Harry S Truman,Democratic
X3OH3J,0,1034,1924,1.609438,Hickenlooper,Smith,1,Warren G. Harding,Republican
X3U0KO,0,2303,1925,1.791759,Story,William,1,Ulysses Grant,Republican
X53HAD,0,1764,1924,0.000000,Nielsen,Leland,1,Richard M. Nixon,Republican
X9VC5V,0,493,1925,0.000000,Connally,Ben,1,Harry S Truman,Democratic


In [10]:
from datetime import datetime
je = datetime.now()
pkl_fname = 'cases_metadata.' + je.strftime('%Y%m%d_%H%M%S' + ".pkl")
md.to_pickle(pkl_fname)


In [11]:
ju.head()

,LastName,FirstName,Gender,CourtType,Pres,Party
judge_id,,,,,,
3563,Abrams,Leslie,0,U.S. District Court,Barack Obama,Democratic
3419,Abrams,Ronnie,0,U.S. District Court,Barack Obama,Democratic
1,Abruzzo,Matthew,1,U.S. District Court,Franklin D. Roosevelt,Democratic
2,Acheson,Marcus,1,U.S. District Court,Rutherford B. Hayes,Republican
3,Acker,William,1,U.S. District Court,Ronald Reagan,Republican
